# Introduction: Testing the Spatial-KWD library
In this notebook, we present a short tutorial on the use of the python wrapper for the **Spatial-KWD** software library.

The goal of this notebook is to show how to use Spatial-KWD to compute the distance between pairs of 2-dimensional histograms.

The only requirements for running this notebook are:

1.   A recent version of [Cython](https://cython.org/) (>= v0.23) installed on your PC.
2.   To Download the source code of the Spatial-KWD python wrapper from [EUROSTAT public repository](https://github.com/eurostat/Spatial-KWD/tree/main/wrappers/python).





In [5]:
import shutil
import sys

# Install Cython
if not shutil.which("cython"):
  !pip install -q cython
  assert(shutil.which("cython"))

# Get the Spatial-KWD source files
if "google.colab" in sys.modules:
  !wget -N -q "https://mate.unipv.it/gualandi/s-kwd.zip"
  !unzip -o -q s-kwd.zip

Once you have the requirements ready, you can compile the library, with the following command

In [6]:
!python setup.py build_ext --inplace

Compiling histogram2D.pyx because it changed.
[1/1] Cythonizing histogram2D.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/histogram2D.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'compiler_directives'
  warnings.warn(msg)
running build_ext
building 'KWD' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I. -I/usr/include/python3.6m -c histogram2D.cpp -o build/temp.linux-x86_64-3.6/histogram2D.o -Wno-unused-function -std=c++14
In file included from KWD_Histogram2D.h:42:0,
                 from KWD_Histogram2D.cpp:9,
                 from histogram2D.cpp:647:
KWD_NetSim

### Histogram2D
The Spatial-KWD library contains a main class of type `Histogram2D` that is used to store a 2-dimensional histogram.
Note that the histogram does not need to be *rectangular*, but it can be defined by specifing the bins coordinates of the elements with a strictly positive weight.

This class has three main methods:

1. A standard costructor `h = Histogram2D()` 
2. A method to add a single point with positive weight: `h.add(int x, int y, double weight)`
3. A method to *normalize* the bin weights, so that the sum of all weights is equal to 1: `h.normalize()`

We now show to use this class to build three different histograms.

In [8]:
from KWD import *

# Define first histogram
a = Histogram2D()
# Add at position (0,0) a unit of mass
a.add(0, 0, 1)
# Normalize the histogram
a.normalize()

# Define second histogram
b = Histogram2D()
# Add at position (1,1) a unit of mass
b.add(1, 1, 2)
# Normalize the histogram
b.normalize()

These two dummy histograms have a single bin with positive weight. The second histogram is normalized and the effect is to reduce its weight to 1.

Then, we add a third histogram having three non empty bins. Please, note that we are not using a regular grid to define this histogram. You are free to position the points at any pair of integer coordinates.

In [9]:
# Define third histogram
c = Histogram2D()
# Add at position (1,0) and (0,1) an half unit of mass
c.add(1, 0, 0.5)
c.add(0, 1, 0.5)
# Add at position (5,5) a unit of mass
c.add(5, 5, 1)
# Normalize the histogram
c.normalize()

After normalizarion, the weights of the three bins become 0.25, 0.25, 0.5.

We show in the next section how to compute the distances among these three histograms. 

### Using the Solver to compute hte KWD distances
The Spatial-KWD library has another object called `Solver` which is used to configure the algorithm that computes the distance between a pair of histograms.

In [11]:
# Create an instance of type Solver 
s = Solver()

Finally we can compute the distance among the three histograms using the method `s.distance(Histogram2D h1, Histogram2D h1, L=3)`.
The method distance first creates a cloud of points considering the union of non empty bins of the two histograms, second it computes the convex hull of all this cloud of points, and finally it computes the Kantorovich-Wasserstein distances between the two histograms using the algorithm described in [[1]](https://epubs.siam.org/doi/abs/10.1137/19M1261195).

The third parameter `L` is used to specify the approximation quality of the algorithm, and it will be described in detail in the second tutorial on this library.


In [12]:
print("d(a,b) =", s.distance(a, b, 3))
print("d(a,c) =", s.distance(a, c, 3))
print("d(b,c) =", s.distance(b, c, 3))

d(a,b) = 1.4142135623730951
d(a,c) = 4.035533905932738
d(b,c) = 2.8284271247461903


Please, if you have any comment on this notebook or on the Spatial-KWD you can drop me an email.

The source code of the library is freely available on GitHub at [Eurostat/Spatial-KWD](https://github.com/eurostat/Spatial-KWD).

### References

[1] Bassetti F., Gualandi S., Veneroni M. (2018): [**On the computation of Kantorovich-Wasserstein distances between 2D-histograms by uncapacitated minimum cost flows**](https://epubs.siam.org/doi/abs/10.1137/19M1261195). SIAM J. Optim., 30(3), 2441–2469, 2020. Preprint on arXiv: [1804.00445](https://arxiv.org/abs/1804.00445).